In [32]:
#The goal is to take a pdf file of all transcripts and convert it to a csv file with the following columns: name of the character and the line they say.

In [33]:
character_names = """

Logan Roy
Kendall Roy
Marcia Roy
Greg Hirsch
Shiv Roy
Roman Roy
Connor Roy
Tom Wambsgans
Frank Vernon
Lawrence Yee
Colin Stiles
Alessandro
Rava
Grace
Trainer
Sarah
Marianne
Pablo
Kid at theme park
Theme-park attendant
Business alchemist
Photographer 1
Leftie (helicopter pilot)
Isla
Sophie Roy
Iverson Roy
Tolly
Logan’s pilot
Kid 2 at theme park
Photographer 2
Kendall’s lawyer
Doctor
Male news anchor
Female news anchor
Airplane passenger
Flight attendant
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
frank vernon
lawrence yee
colin stiles
rava
sarah
marianne
paramedic alex long
doctor kenneth warner
nurse carrie buckley
nurse teresa menendez
doctor quinn ashbury
doctor rashida johnson
jess jordan
gerri kellman
eva
karolina novotney
karl muller
doctor thomas lewis
neurologist jeremy lipe
willa ferreyra
doorman
nurse elisa highsmith
jai
newscaster
doctor val montgomery
joyce miller
doctor troy judith
fikret
tatsuya
puking guy
halal guy
greg’s taxi driver
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
frank vernon
lawrence yee
colin stiles
rava
sarah
marianne
sophie roy
iverson roy
joan
brex
lisa
gerri kellman
karl muller
jess jordan
polk
receptionist
eva
karolina novotney
sarah
daniel miller
male executive
stewy hosseini
photographer
nate sofrelli
sandy furness
hipster clerk
malaya
brad kilmartin
doorman
fiona
gladhander 1
gladhander 2
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
frank vernon
colin stiles
grace o’hare
sarah
gerri kellman
joyce miller
daniel miller
jai
karolina novotney
stewy hosseini
doctor troy judith
willa ferreyra
eva
jess jordan
joan
karl muller
lucy
roland
karen
anna newman
mark ravenhead
stephanie
elevator guy
teleprompter operator
erin
bill
kelly
waitress
waiter
dancer
jeane
dance troupe 1
dance troupe 2
dance troupe 3
dance troupe 4
dance troupe 5
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
rava
karl muller
grace o’hare
marianne
sophie roy
iverson roy
isla
gerri kellman
willa ferreyra
ewan roy
brex
young executive 1
young executive 2
chris
greta
amir
office keeper
uber driver
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
frank vernon
lawrence yee
colin stiles
karl muller
gerri kellman
ewan roy
joyce miller
stewy hosseini
jess jordan
kalfu
nate sofrelli
aide 1
aide 2
waiter
sarita shenoy
richard
tatsuya
ilona shenoy
pilot
driver
paul chambers
dewi swann
datu
jeane
sarah
asha
noah
logan roy
kendall roy
marcia roy
shiv roy
roman roy
connor roy
tom wambsgans
rava
colin stiles
stewy hosseini
gerri kellman
karl muller
karolina novotney
willa ferreyra
nate sofrelli
gil eavis
mrs wambsgans
mr wambsgans
tabitha
caroline collingwood
charlotte
matt
jonas
barmaid
vicar
doddy
logan roy
kendall roy
marcia roy
greg hirsch
shiv roy
roman roy
connor roy
tom wambsgans
rava
frank vernon
colin stiles
sophie roy
iverson roy
jess jordan
gerri kellman
willa ferreyra
stewy hosseini
sandy furness
nate sofrelli
gil eavis
mrs wambsgans
mr wambsgans
tabitha
caroline collingwood
amir
charlotte
matt
jonas
denis
doddy
photographer
"""

character_list = [name.strip().upper() for name in character_names.split('\n') if name.strip()]
character_list = list(dict.fromkeys(character_list)) # Remove duplicates

In [13]:
import csv
import re
import fitz  # PyMuPDF
import pandas as pd 
from bs4 import BeautifulSoup
import os

# Open the PDF file
pdf_document = fitz.open("C:/Users/a/Desktop/Programmation/TV Series Analysis Project/Data/Transcripts.pdf")
full_text = ""



# Extract text and formatting information
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
        if 'lines' in block:  # Check if 'lines' key exists
            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"]
                    font_size = span["size"]
                    
                    # Check if the text is italic
                    if span["flags"] & 2:
                        text = f"<i>{text}</i>"
                    
                    # Check if the text is in a larger font size
                    if font_size > 12:  # Adjust the threshold as needed
                        text = f"<b>{text}</b>"
                    
                    full_text += text + "\n"

#Delete the first 679 lines of the text
full_text = full_text.split("\n")
full_text = full_text[679:]

#Delete all the italic elements 
full_text = [line for line in full_text if "<i>" not in line]

#Delete lines with just parentheses
full_text = [line for line in full_text if "(" not in line]
full_text = [line for line in full_text if ")" not in line]


#Delete lines in UPPERCASE (not starting by <b>) not in the character list:
full_text = [line for line in full_text if line.startswith('<b>') or any(char_name.startswith(line) for char_name in character_list)]

# Initialize an empty list to store the data
data = []

# Variables to keep track of the current character and their lines
current_character = None
current_lines = []

# Iterate over the full text
for line in full_text:
    if line.isupper() and not line.startswith('<b>'):
        # If we encounter a new character, save the previous character and their lines
        if current_character is not None:
            data.append([current_character, ' '.join(current_lines)])
        
        # Update the current character and reset their lines
        current_character = line
        current_lines = []
    else:
        # Add the line to the current character's lines
        current_lines.append(line)

# Append the last character and their lines
if current_character is not None:
    data.append([current_character, ' '.join(current_lines)])

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=['Character', 'Line'])


FileNotFoundError: no such file: 'TV Series Analysis Project/Data/Transcripts.pdf'

In [35]:
# Function to clean HTML tags and calculate length
def clean(line):
    cleaned_line = BeautifulSoup(line, "html.parser").get_text()
    return cleaned_line


# Replace the data frame number (5855) because it take metadata at then end of the pdf
df["Line"][5855]="<b>This could be the defining event of your life. It’d eat everything. A</b> <b>rich kid kills a boy. You’ll never be anything else. Or—</b> <b>Or it could be what it should be. Nothing at all. A sad little detail at a</b> <b>lovely wedding where father and son were reconciled?</b> <b>You’re a good kid. You could still be a good kid.</b>"

#Clean the lines
df["Line"]=df["Line"].apply(clean)

#Delete the lines with more 700 characters
df = df[df["Line"].str.len() < 600]

# Print the updated DataFrame
print(df)




     Character                                               Line
0        LOGAN                   Where am I? Where the fuck am I?
1       MARCIA                           Logan? It’s okay, Logan.
2        LOGAN                                   Ugh. Okay. Okay.
3       MARCIA                                         It’s okay.
4        LOGAN                                        Where am I?
...        ...                                                ...
5851     LOGAN                  Yeah. I think that would be good.
5852   KENDALL  Um. Right. I mean, nobody did anything wrong a...
5853     LOGAN  Tell Sandy you’re out. Tell Stewy. The thing l...
5854   KENDALL  There’s nothing— I don’t think. I wasn’t there...
5855     LOGAN  This could be the defining event of your life....

[5840 rows x 2 columns]


In [36]:
# Save the DataFrame to a CSV file with two columns: Character and Line
df.to_csv('transcripts.csv', index=False)
df.to_excel('transcripts.xlsx', index=False)